<a href="https://colab.research.google.com/github/UmarFarooqui/Springboard/blob/master/Data_Wrangling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from pprint import pprint
import json
from pathlib import Path
import pandas as pd
import requests
from google.colab import files

In [4]:
# Mounting the google drive
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [5]:
# Uploaded the dataset to google drive. Checking.
!ls "/content/drive/My Drive/data"

codesearchnet_python.zip


In [6]:
!unzip "/content/drive/My Drive/data/codesearchnet_python.zip"

Archive:  /content/drive/My Drive/data/codesearchnet_python.zip
   creating: python/
   creating: python/final/
   creating: python/final/jsonl/
   creating: python/final/jsonl/train/
  inflating: python/final/jsonl/train/python_train_9.jsonl.gz  
  inflating: python/final/jsonl/train/python_train_12.jsonl.gz  
  inflating: python/final/jsonl/train/python_train_10.jsonl.gz  
  inflating: python/final/jsonl/train/python_train_0.jsonl.gz  
  inflating: python/final/jsonl/train/python_train_6.jsonl.gz  
  inflating: python/final/jsonl/train/python_train_2.jsonl.gz  
  inflating: python/final/jsonl/train/python_train_4.jsonl.gz  
  inflating: python/final/jsonl/train/python_train_8.jsonl.gz  
  inflating: python/final/jsonl/train/python_train_11.jsonl.gz  
  inflating: python/final/jsonl/train/python_train_5.jsonl.gz  
  inflating: python/final/jsonl/train/python_train_13.jsonl.gz  
  inflating: python/final/jsonl/train/python_train_3.jsonl.gz  
  inflating: python/final/jsonl/train/python

In [0]:
# Exploring the full dataset
python_files = sorted(Path('python/').glob('**/*.gz'))

In [8]:
python_files

[PosixPath('python/final/jsonl/test/python_test_0.jsonl.gz'),
 PosixPath('python/final/jsonl/train/python_train_0.jsonl.gz'),
 PosixPath('python/final/jsonl/train/python_train_1.jsonl.gz'),
 PosixPath('python/final/jsonl/train/python_train_10.jsonl.gz'),
 PosixPath('python/final/jsonl/train/python_train_11.jsonl.gz'),
 PosixPath('python/final/jsonl/train/python_train_12.jsonl.gz'),
 PosixPath('python/final/jsonl/train/python_train_13.jsonl.gz'),
 PosixPath('python/final/jsonl/train/python_train_2.jsonl.gz'),
 PosixPath('python/final/jsonl/train/python_train_3.jsonl.gz'),
 PosixPath('python/final/jsonl/train/python_train_4.jsonl.gz'),
 PosixPath('python/final/jsonl/train/python_train_5.jsonl.gz'),
 PosixPath('python/final/jsonl/train/python_train_6.jsonl.gz'),
 PosixPath('python/final/jsonl/train/python_train_7.jsonl.gz'),
 PosixPath('python/final/jsonl/train/python_train_8.jsonl.gz'),
 PosixPath('python/final/jsonl/train/python_train_9.jsonl.gz'),
 PosixPath('python/final/jsonl/valid/p

In [9]:
print(f'Total number of files: {len(python_files)}')

Total number of files: 16


In [0]:
columns_list = ['repo', 'path', 'url', 'func_name', 'code', 'code_tokens', 'docstring', 'docstring_tokens']

In [0]:
def jsonl_list_to_dataframe(file_list, columns=columns_list):
  """Load a list of jsonl.gz files into a pandas dataframe."""
  return pd.concat([pd.read_json(f, orient='records', compression='gzip', lines=True)[columns] for f in file_list], sort=False) 

In [0]:
pydf = jsonl_list_to_dataframe(python_files)

In [14]:
pydf.head(3)

,repo,path,url,func_name,code,code_tokens,docstring,docstring_tokens
0,soimort/you-get,src/you_get/extractors/youtube.py,https://github.com/soimort/you-get/blob/b746ac...,YouTube.get_vid_from_url,"def get_vid_from_url(url):\n """"""Extract...","[def, get_vid_from_url, (, url, ), :, return, ...",Extracts video ID from URL.,"[Extracts, video, ID, from, URL, .]"
1,soimort/you-get,src/you_get/extractors/miomio.py,https://github.com/soimort/you-get/blob/b746ac...,sina_xml_to_url_list,"def sina_xml_to_url_list(xml_data):\n """"""st...","[def, sina_xml_to_url_list, (, xml_data, ), :,...",str->list\n Convert XML to URL List.\n F...,"[str, -, >, list, Convert, XML, to, URL, List,..."
2,soimort/you-get,src/you_get/extractors/fc2video.py,https://github.com/soimort/you-get/blob/b746ac...,makeMimi,"def makeMimi(upid):\n """"""From http://cdn37....","[def, makeMimi, (, upid, ), :, strSeed, =, ""gG...",From http://cdn37.atwikiimg.com/sitescript/pub...,"[From, http, :, //, cdn37, ., atwikiimg, ., co..."


In [15]:
# The columns which will be used for our purpose would be func_name, code, code_tokens, docstring, docstring_tokens
pydf.groupby(['repo'])['code'].count()

repo
0101/pipetools                   12
02strich/django-auth-kerberos     1
05bit/peewee-async               42
0compute/xtraceback               2
0k/kids.cache                     4
                                 ..
zyga/python-glibc                32
zyga/python-phablet               6
zzyztyy/pyIGRF                    6
zzzsochi/includer                 2
zzzsochi/resolver_deco            1
Name: code, Length: 13590, dtype: int64

In [16]:
# As a starting point we will use the repos with a high star count
# Example of how to do it
url = 'http://api.github.com/repos/'
repo_name = '0101/pipetools'
r = requests.get(url + repo_name)
r_json = r.json()
r_json['stargazers_count']

81

In [17]:
# Now we will add this as the last column in our dataframe
# First check if we have no null values in the repo column
pydf.repo.isnull().any()

False

In [18]:
pydf[pydf['repo'] == '']

,repo,path,url,func_name,code,code_tokens,docstring,docstring_tokens


In [19]:
# So looks like the repo column is having no null values or empty string
# Taking a look at the dataframe
pydf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 457461 entries, 0 to 23106
Data columns (total 8 columns):
repo                457461 non-null object
path                457461 non-null object
url                 457461 non-null object
func_name           457461 non-null object
code                457461 non-null object
code_tokens         457461 non-null object
docstring           457461 non-null object
docstring_tokens    457461 non-null object
dtypes: object(8)
memory usage: 31.4+ MB


In [20]:
# Looks like we have no null values
# Length of the dataframe
len(pydf)

457461

In [21]:
# So we have 0.45M samples, we will increase our dataset if we need to by exploring more repos.
unique_repo_name_series = pydf.repo.drop_duplicates()
unique_repo_name_series

0                       soimort/you-get
52                       apache/airflow
849                      pytorch/vision
900                      asciimoo/searx
910              tensorflow/probability
                      ...              
23018           praekelt/python-ambient
23024                   zenreach/py-era
23090    TakesxiSximada/custom_settings
23095              openpermissions/bass
23099                 xnuinside/clifier
Name: repo, Length: 13590, dtype: object

In [23]:
# Turns out that for API request, we can only make up to 5000 request per hour
# So, I will not do this for now and explore other means to get the star information
repo_star_map = {}
first_4000_unique_repo_name_series = unique_repo_name_series[:4000]
for index, repo_name in first_4000_unique_repo_name_series.items():
  r = requests.get(url + repo_name)
  r_json = r.json()
  if r_json['stargazers_count']:
    repo_star_map[repo_name] = r_json['stargazers_count']

KeyError: ignored

In [24]:
repo_star_map

{'3DLIRIOUS/MeshLabXML': 117,
 'Azure/azure-sdk-for-python': 909,
 'CITGuru/PyInquirer': 561,
 'Danielhiversen/PyXiaomiGateway': 102,
 'Microsoft/botbuilder-python': 265,
 'Nekmo/amazon-dash': 567,
 'PyCQA/pylint': 2417,
 'Qiskit/qiskit-terra': 2634,
 'SectorLabs/django-postgres-extra': 254,
 'Tivix/django-rest-auth': 1848,
 'alpacahq/alpaca-trade-api-python': 224,
 'ansible/ansible-lint': 1859,
 'apache/airflow': 14535,
 'asciimoo/searx': 5598,
 'bloomreach/s4cmd': 929,
 'bluedisk/hangul-toolkit': 154,
 'celiao/tmdbsimple': 250,
 'chaoss/grimoirelab-perceval': 140,
 'dagster-io/dagster': 983,
 'deepmipt/DeepPavlov': 3668,
 'ekzhu/datasketch': 1087,
 'geopython/OWSLib': 187,
 'google/brotli': 7916,
 'google/gin-config': 581,
 'graphql-python/graphene-sqlalchemy': 603,
 'h2oai/h2o-3': 4434,
 'howie6879/ruia': 1028,
 'instagrambot/instabot': 2705,
 'intel-analytics/BigDL': 3182,
 'kennethreitz/twitter-scraper': 1886,
 'lensacom/sparkit-learn': 989,
 'lepture/flask-oauthlib': 1384,
 'libr

In [0]:
# Persisting the dataframe containing star information for the repositories
with open('repository_star_info.json', 'w') as outfile:
  json.dump(repo_star_map, outfile)
files.download('repository_star_info.json')

In [37]:
# Uploaded the downloaded file to the gdrive, checking if the system recognizes it
!ls "/content/drive/My Drive/data"

codesearchnet_python.zip  repository_star_info.json


In [38]:
# Reading the json from the file
with open('repository_star_info.json') as json_file:
  repository_metadata_info = json.load(json_file)
repository_metadata_info

{'3DLIRIOUS/MeshLabXML': 117,
 'Azure/azure-sdk-for-python': 909,
 'CITGuru/PyInquirer': 561,
 'Danielhiversen/PyXiaomiGateway': 102,
 'Microsoft/botbuilder-python': 265,
 'Nekmo/amazon-dash': 567,
 'PyCQA/pylint': 2417,
 'Qiskit/qiskit-terra': 2634,
 'SectorLabs/django-postgres-extra': 254,
 'Tivix/django-rest-auth': 1848,
 'alpacahq/alpaca-trade-api-python': 224,
 'ansible/ansible-lint': 1859,
 'apache/airflow': 14535,
 'asciimoo/searx': 5598,
 'bloomreach/s4cmd': 929,
 'bluedisk/hangul-toolkit': 154,
 'celiao/tmdbsimple': 250,
 'chaoss/grimoirelab-perceval': 140,
 'dagster-io/dagster': 983,
 'deepmipt/DeepPavlov': 3668,
 'ekzhu/datasketch': 1087,
 'geopython/OWSLib': 187,
 'google/brotli': 7916,
 'google/gin-config': 581,
 'graphql-python/graphene-sqlalchemy': 603,
 'h2oai/h2o-3': 4434,
 'howie6879/ruia': 1028,
 'instagrambot/instabot': 2705,
 'intel-analytics/BigDL': 3182,
 'kennethreitz/twitter-scraper': 1886,
 'lensacom/sparkit-learn': 989,
 'lepture/flask-oauthlib': 1384,
 'libr

In [39]:
# Append a column in the pydf dataframe called star that contains the star information we have discovered so far
pydf.head(3)

,repo,path,url,func_name,code,code_tokens,docstring,docstring_tokens
0,soimort/you-get,src/you_get/extractors/youtube.py,https://github.com/soimort/you-get/blob/b746ac...,YouTube.get_vid_from_url,"def get_vid_from_url(url):\n """"""Extract...","[def, get_vid_from_url, (, url, ), :, return, ...",Extracts video ID from URL.,"[Extracts, video, ID, from, URL, .]"
1,soimort/you-get,src/you_get/extractors/miomio.py,https://github.com/soimort/you-get/blob/b746ac...,sina_xml_to_url_list,"def sina_xml_to_url_list(xml_data):\n """"""st...","[def, sina_xml_to_url_list, (, xml_data, ), :,...",str->list\n Convert XML to URL List.\n F...,"[str, -, >, list, Convert, XML, to, URL, List,..."
2,soimort/you-get,src/you_get/extractors/fc2video.py,https://github.com/soimort/you-get/blob/b746ac...,makeMimi,"def makeMimi(upid):\n """"""From http://cdn37....","[def, makeMimi, (, upid, ), :, strSeed, =, ""gG...",From http://cdn37.atwikiimg.com/sitescript/pub...,"[From, http, :, //, cdn37, ., atwikiimg, ., co..."


In [40]:
pydf_with_star = pydf
pydf_with_star['star'] = ""
for index, row in pydf_with_star.iterrows():
  if row['repo'] in repository_metadata_info:
    row['star'] = repository_metadata_info[row['repo']]
pydf_with_star

,repo,path,url,func_name,code,code_tokens,docstring,docstring_tokens,star
0,soimort/you-get,src/you_get/extractors/youtube.py,https://github.com/soimort/you-get/blob/b746ac...,YouTube.get_vid_from_url,"def get_vid_from_url(url):\n """"""Extract...","[def, get_vid_from_url, (, url, ), :, return, ...",Extracts video ID from URL.,"[Extracts, video, ID, from, URL, .]",27926
1,soimort/you-get,src/you_get/extractors/miomio.py,https://github.com/soimort/you-get/blob/b746ac...,sina_xml_to_url_list,"def sina_xml_to_url_list(xml_data):\n """"""st...","[def, sina_xml_to_url_list, (, xml_data, ), :,...",str->list\n Convert XML to URL List.\n F...,"[str, -, >, list, Convert, XML, to, URL, List,...",27926
2,soimort/you-get,src/you_get/extractors/fc2video.py,https://github.com/soimort/you-get/blob/b746ac...,makeMimi,"def makeMimi(upid):\n """"""From http://cdn37....","[def, makeMimi, (, upid, ), :, strSeed, =, ""gG...",From http://cdn37.atwikiimg.com/sitescript/pub...,"[From, http, :, //, cdn37, ., atwikiimg, ., co...",27926
3,soimort/you-get,src/you_get/extractors/fc2video.py,https://github.com/soimort/you-get/blob/b746ac...,fc2video_download,"def fc2video_download(url, output_dir = '.', m...","[def, fc2video_download, (, url, ,, output_dir...",wrapper,[wrapper],27926
4,soimort/you-get,src/you_get/extractors/dailymotion.py,https://github.com/soimort/you-get/blob/b746ac...,dailymotion_download,"def dailymotion_download(url, output_dir='.', ...","[def, dailymotion_download, (, url, ,, output_...",Downloads Dailymotion videos by URL.,"[Downloads, Dailymotion, videos, by, URL, .]",27926
...,...,...,...,...,...,...,...,...,...
23102,xnuinside/clifier,clifier/clifier.py,https://github.com/xnuinside/clifier/blob/3d70...,Clifier.show_version,"def show_version(self):\n """""" custom co...","[def, show_version, (, self, ), :, class, Show...",custom command line action to show version,"[custom, command, line, action, to, show, vers...",
23103,xnuinside/clifier,clifier/clifier.py,https://github.com/xnuinside/clifier/blob/3d70...,Clifier.check_path_action,"def check_path_action(self):\n """""" cust...","[def, check_path_action, (, self, ), :, class,...",custom command line action to check file exist,"[custom, command, line, action, to, check, fil...",
23104,tklovett/PyShirtsIO,interactive_console.py,https://github.com/tklovett/PyShirtsIO/blob/ff...,new_user,def new_user(yaml_path):\n '''\n Return ...,"[def, new_user, (, yaml_path, ), :, print, 'Re...",Return the consumer and oauth tokens with thre...,"[Return, the, consumer, and, oauth, tokens, wi...",
23105,ibelie/typy,typy/google/protobuf/internal/python_message.py,https://github.com/ibelie/typy/blob/3616845fb9...,_AddPropertiesForExtensions,"def _AddPropertiesForExtensions(descriptor, cl...","[def, _AddPropertiesForExtensions, (, descript...",Adds properties for all fields in this protoco...,"[Adds, properties, for, all, fields, in, this,...",
